In [5]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")
# initialize pinecone
pinecone.init(
    api_key=os.getenv("d394083d-4595-4d7d-823f-86fc0b659774"),  # find at app.pinecone.io
    environment=os.getenv("gcp-starter"),  # next to api key in console
)

index_name = "hackathon"

In [65]:
from langchain_community.document_loaders import CSVLoader
import os
from langchain_community.llms import Clarifai
from langchain_community.embeddings import HuggingFaceEmbeddings
import getpass
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from clarifai.client.model import Model
from clarifai.client.input import Inputs
from langchain_community.vectorstores import Chroma
import chromadb
import time
from pydub import AudioSegment
from pydub.playback import play
import io
import base64


In [2]:
# Create embeddings and the Chromadb vector store
embeddings = HuggingFaceEmbeddings(model_name="WhereIsAI/UAE-Large-V1")

c:\Users\zibaa\OneDrive\Desktop\EmpoweredRecovery\v\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name C:\Users\zibaa/.cache\torch\sentence_transformers\WhereIsAI_UAE-Large-V1. Creating a new one with MEAN pooling.


In [3]:
os.environ["CLARIFAI_PAT"]="9a6cd1a04c8c460ba8e27e7db99b4e4e"
MODEL_URL="https://clarifai.com/openai/chat-completion/models/gpt-4-turbo"
llm = Clarifai(model_url=MODEL_URL)
SPEECH_URL = 'https://clarifai.com/openai/tts/models/openai-tts-1'
tts = Clarifai(model_url=SPEECH_URL)

loader = CSVLoader(
    file_path="meditation-scripts.csv", source_column="Text")
data = loader.load()

In [ ]:
# Specify the Deep Lake dataset path
dataset_path = "chromadb_uae"  # Replace with your desired path
# Create embeddings and the Chromadb vector store
db = Chroma.from_documents(documents=data, embedding=embeddings, persist_directory=dataset_path, 
    collection_name="meditation")

In [7]:
client = chromadb.PersistentClient("chromadb_uae")  # Specify data path
print(client.list_collections())

[Collection(name=meditation)]


In [8]:
collection = db.get()


In [9]:
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=db.as_retriever())

In [30]:
template = '''
You are an exceptional meditation guide for people recovering from addiction.
You generate responses based only on the Text content of the retrieved documents.
You start by greeting the user by {user_name} in a freiendly and professional manner.
 
Provide a meditation script based on the user input for meditation theme by using the following context information.
{query} themed script for a {user_age}-year-old {user_gender}, struggling with {user_struggle}, and currently feels like {user_emotion}.

Adapt the length of the script based on the {meditation_length}. 
If {meditation_length}>5 minutes, lengthen the meditation script by:
    Deepening focus: Explore breath details, mantra variations, or body scan depth.
    Expanding imagery: Paint vivid pictures, create a calming narrative.
    Adding prompts: Encourage reflection, ask gentle questions

If {meditation_length}>10 minutes, lengthen the meditation script by:
    Deepening focus: Explore breath details, mantra variations, or body scan depth.
    Expanding imagery: Paint vivid pictures, create a calming narrative.
    Adding prompts: Encourage reflection, ask gentle questions.
    Layering techniques: Combine breath with visualizations, scans, affirmations.
    Adding pauses/reiterations: Allow absorption, reinforce key points.

Begin from the meditation script directly.
'''
prompt = PromptTemplate(
    input_variables = ["user_name", "query", "user_age",
                        "user_gender", "user_struggle",
                         "user_emotion", "meditation_length"],
    template=template
)

In [34]:
# user question
user_name = "Nyx"
user_age = 25
user_gender = "female"
user_struggle = "I struggle with feeling confident in myself to overcome my constant battle with addiction. I feel like I may slip up and never get back up again."
user_emotion = "I am very overwhelmed today. I feel like I have too much happening and my body and mind are not working with me."
query = "Mindfulness and Relaxation"
meditation_length = "5 minutes"
# Create the prompt using the template and user input
user_prompt = prompt.format(
    user_name = user_name,
    query = query,
    user_age = user_age,
    user_gender = user_gender,
    user_struggle = user_struggle,
    user_emotion = user_emotion,
    meditation_length = meditation_length
)

# Use the chain to get a meditation script based on the user input
meditation_script1 = chain.invoke(user_prompt)

# Print or use the meditation script as needed
print(meditation_script1["result"])

Good evening, Nyx. Let's take a journey together through mindfulness and relaxation. As you settle in, allow yourself to be present in this moment, letting go of the day's overwhelm as we embark on a path to inner peace and confidence.

Begin by finding a comfortable position, sitting or lying down. Close your eyes gently, and let's start with some deep breaths to ground ourselves. Inhale slowly through your nose, feeling your chest and belly expand with fresh air. Exhale through your mouth, releasing any tension you might be holding. With each breath, feel yourself becoming more relaxed.

As you continue to breathe, allow my words to guide you: "I am one with the serenity of nature. Each breath connects me with the calming energy of the outdoors. In this moment, I find peace in nature's embrace."

Visualize yourself in a tranquil forest. The trees around you are tall and sturdy, their leaves whispering the songs of the earth. You are not alone; nature is with you, supporting you. Feel

In [32]:
# user question
user_name = "Nyx"
user_age = 25
user_gender = "female"
user_struggle = "I struggle with feeling confident in myself to overcome my constant battle with addiction. I feel like I may slip up and never get back up again."
user_emotion = "I am very overwhelmed today. I feel like I have too much happening and my body and mind are not working with me."
query = "Mindfulness and Relaxation"
meditation_length = "10 minutes"
# Create the prompt using the template and user input
user_prompt = prompt.format(
    user_name = user_name,
    query = query,
    user_age = user_age,
    user_gender = user_gender,
    user_struggle = user_struggle,
    user_emotion = user_emotion,
    meditation_length = meditation_length
)

# Use the chain to get a meditation script based on the user input
meditation_script2 = chain.invoke(user_prompt)

# Print or use the meditation script as needed
print(meditation_script2["result"])

Welcome, Nyx. Let's begin a Mindfulness and Relaxation meditation to help you find balance and confidence on your journey to recovery.

Begin by finding a comfortable position, sitting or lying down, in a place where you can be undisturbed for the next while. Close your eyes if you feel comfortable doing so, and let's take a moment to arrive in this space, together.

Take a deep breath in, filling your lungs with fresh air. Exhale slowly, feeling any tension melting away from your body. Inhale again, deeply, and as you exhale, visualize releasing the overwhelming thoughts and worries that have been holding you tight.

With each inhale, imagine drawing in strength and confidence. With every exhale, let go of the fear of slipping up. Remember, you are worthy of peace and healing. Your journey is a path of self-discovery and growth.

Let's deepen our focus. Pay attention to the sensation of the breath as it enters through your nostrils, fills your chest and belly, and then releases back i

In [33]:
# user question
user_name = "Nyx"
user_age = 25
user_gender = "female"
user_struggle = "I struggle with feeling confident in myself to overcome my constant battle with addiction. I feel like I may slip up and never get back up again."
user_emotion = "I am very overwhelmed today. I feel like I have too much happening and my body and mind are not working with me."
query = "Mindfulness and Relaxation"
meditation_length = "15 minutes"
# Create the prompt using the template and user input
user_prompt = prompt.format(
    user_name = user_name,
    query = query,
    user_age = user_age,
    user_gender = user_gender,
    user_struggle = user_struggle,
    user_emotion = user_emotion,
    meditation_length = meditation_length
)

# Use the chain to get a meditation script based on the user input
meditation_script3 = chain.invoke(user_prompt)

# Print or use the meditation script as needed
print(meditation_script3["result"])

Welcome, Nyx. Let's begin our mindfulness and relaxation meditation to help you find confidence and calm in your journey towards recovery.

Begin by finding a comfortable seated position. Close your eyes, and allow yourself to arrive fully in this moment. Take a deep breath in, filling your lungs with air, and exhale slowly, letting go of any immediate worries or stress. With each breath, feel yourself becoming more relaxed.

Now, focus on your breath. Notice the sensation of the air entering through your nostrils, filling your chest and belly, and then gently leaving your body. Imagine with each inhalation you are drawing in strength and serenity, and with each exhalation, you are releasing doubt and fear.

Repeat after me: "I am one with the serenity of nature. Each breath connects me with the calming energy of the outdoors. In this moment, I find peace in nature's embrace."

Allow this affirmation to resonate within you. Visualize yourself surrounded by nature, perhaps a tranquil fo

In [43]:
type(meditation_script3["result"])

str

In [51]:
sections = meditation_script1["result"].split("\n\n")
type(sections)

list

In [54]:
def get_pause_duration(section_index):
    if meditation_length == "5 minutes":
        durations = {
            0: 2,  # 5 seconds before section 1
            1: 5,
            2: 10,
            3: 15,
            4: 20,
            5: 20,
            6: 20,
            7: 10,
            8: 10,
            9: 5
        }
    elif meditation_length == "10 minutes":
        durations = {
            0: 2,  # 5 seconds before section 1
            1: 5,
            2: 15,
            3: 20,
            4: 25,
            5: 30,
            6: 30,
            7: 30,
            8: 30,
            9: 15
        }
    else:
        durations = {
            0: 2,  # 5 seconds before section 1
            1: 5,
            2: 20,
            3: 25,
            4: 30, 
            5: 35,
            6: 35,
            7: 35,
            8: 35,
            9: 30
        }
    return durations.get(section_index, 3)
    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:47                                                                                   │
│                                                                                                  │
│   44                                                                                             │
│   45 for i, section in enumerate(sections):                                                      │
│   46 │   message = {"text": section.encode().decode()}  # Convert bytes to str                   │
│ ❱ 47 │   audio = tts.invoke([message], input_type="text", inference_params={'voice': "onyx",     │
│   48 │   audio.stream_to_file(f"section_{i+1}.mp3")  # Save each section separately              │
│   49 │                                                                                           │
│   50 │   # Pause for a specific duration before the next section                                 │
│                                                                                                  │
│ c:\Users\zibaa\OneDrive\Desktop\EmpoweredRecovery\v\Lib\site-packages\langchain_core\language_mo │
│ dels\llms.py:231 in invoke                                                                       │
│                                                                                                  │
│    228 │   │   config = ensure_config(config)                                                    │
│    229 │   │   return (                                                                          │
│    230 │   │   │   self.generate_prompt(                                                         │
│ ❱  231 │   │   │   │   [self._convert_input(input)],                                             │
│    232 │   │   │   │   stop=stop,                                                                │
│    233 │   │   │   │   callbacks=config.get("callbacks"),                                        │
│    234 │   │   │   │   tags=config.get("tags"),                                                  │
│                                                                                                  │
│ c:\Users\zibaa\OneDrive\Desktop\EmpoweredRecovery\v\Lib\site-packages\langchain_core\language_mo │
│ dels\llms.py:213 in _convert_input                                                               │
│                                                                                                  │
│    210 │   │   elif isinstance(input, str):                                                      │
│    211 │   │   │   return StringPromptValue(text=input)                                          │
│    212 │   │   elif isinstance(input, Sequence):                                                 │
│ ❱  213 │   │   │   return ChatPromptValue(messages=input)                                        │
│    214 │   │   else:                                                                             │
│    215 │   │   │   raise ValueError(                                                             │
│    216 │   │   │   │   f"Invalid input type {type(input)}. "                                     │
│                                                                                                  │
│ c:\Users\zibaa\OneDrive\Desktop\EmpoweredRecovery\v\Lib\site-packages\langchain_core\load\serial │
│ izable.py:107 in __init__                                                                        │
│                                                                                                  │
│   104 │   _lc_kwargs = PrivateAttr(default_factory=dict)                                         │
│   105 │                                                                                          │
│   106 │   def __init__(self, **kwargs: Any) -> None:                                             │
│ ❱ 107 │   │   super().__init__(**kwargs)                   

In [101]:
def inference(model_url, inputs, params={}):
    prediction = Model(model_url).predict(inputs=[inputs], inference_params=params)
    return prediction.outputs[0].data

def text_to_speech(text: str):
    model_url = 'https://clarifai.com/openai/tts/models/openai-tts-1'
    inputs = Inputs.get_text_input(input_id="", raw_text=text)
    data = inference(model_url, inputs, {'voice': 'onyx', 'speed': 1})

    audio_bytes = io.BytesIO(data.audio.base64)

    # Save the audio to a file
    with open("complete_meditation.mp3", "wb") as f:
        f.write(audio_bytes.getvalue())

    return "complete_meditation.mp3"

In [102]:
# Call the text_to_speech function
generated_audio = text_to_speech(meditation_script3['result'])

In [110]:
import wave

def get_pause_duration(section_index, meditation_length):
    durations = {
        "5 minutes": [2, 5, 10, 15, 20, 20, 20, 10, 10, 5],
        "10 minutes": [2, 5, 15, 20, 25, 30, 30, 30, 30, 15],
        "15 minutes": [2, 5, 20, 25, 30, 35, 35, 35, 35, 30],
    }
    return durations.get(meditation_length, [3])[section_index % len(durations.get(meditation_length, [3]))]

def inference(model_url, inputs, params={}):
    prediction = Model(model_url).predict(inputs=[inputs], inference_params=params)
    return prediction.outputs[0].data

def generate_silence(duration, sampling_rate=44100, sample_width=2):
    num_frames = int(duration * sampling_rate)
    silence_data = b'\0' * (num_frames * sample_width)
    return silence_data

def text_to_speech(text: str, meditation_length):
    model_url = 'https://clarifai.com/openai/tts/models/openai-tts-1'
    
    # Split the text into paragraphs
    paragraphs = text.split("\n\n")
    
    # Initialize an empty bytearray to store the complete audio script
    complete_script = bytearray()

    total_duration = 0  # Track the total duration of the generated audio

    for i, paragraph in enumerate(paragraphs):
        # Skip empty paragraphs
        if paragraph.strip():
            # Generate audio for each paragraph
            inputs = Inputs.get_text_input(input_id="", raw_text=paragraph)
            data = inference(model_url, inputs, {'voice': 'onyx', 'speed': 1})

            audio_bytes = io.BytesIO(data.audio.base64)
            audio_duration = len(audio_bytes.getvalue()) / (44100 * 2)  # Duration in seconds (44100 samples per second, 2 bytes per sample)

            # Add the audio content to the complete script
            complete_script.extend(audio_bytes.getvalue())
            
            # Add pause duration between sections
            pause_duration = get_pause_duration(i, meditation_length)
            silence_data = generate_silence(pause_duration, sampling_rate=44100, sample_width=2)
            complete_script.extend(silence_data)

            # Update total duration
            total_duration += audio_duration + pause_duration

    # Save the complete audio script to a file
    with wave.open("complete_meditation.mp3", "wb") as wave_file:
        wave_file.setnchannels(1)  # Mono audio
        wave_file.setsampwidth(2)  # 2 bytes per sample
        wave_file.setframerate(44100)  # 44100 samples per second
        wave_file.writeframes(complete_script)

    return "complete_meditation.mp3"

In [111]:
# Call the text_to_speech function
generated_audio = text_to_speech(meditation_script3['result'], meditation_length="15 minutes")

In [104]:
def generate_silence_sample(duration, sampling_rate=44100):
    num_frames = int(duration * sampling_rate)
    silence_data = b'\0' * (num_frames * 2)  # 2 bytes per frame for 16-bit audio
    return silence_data

# Create a 2-second silence sample
silence_duration = 2  # 2 seconds
silence_sample = generate_silence_sample(silence_duration)

# Save the silence sample as a WAV file
with wave.open("silence_sample.wav", "wb") as wave_file:
    wave_file.setnchannels(1)  # Mono audio
    wave_file.setsampwidth(2)  # 2 bytes per sample
    wave_file.setframerate(44100)  # 44100 samples per second
    wave_file.writeframes(silence_sample)

In [19]:
# user question
user_name = "Nyx"
query = "Give me a relaxing meditation script for addiction."
meditation_length = "10 minutes"
# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(user_name=user_name, chunks_formatted=chunks_formatted, meditation_length=meditation_length, query=query)
answer = llm(prompt_formatted)
print(answer)

Welcome, Nyx. Let's begin a 10-minute relaxing meditation session to aid in your recovery from addiction. Find a quiet, comfortable place where you can sit or lie down without interruptions. Close your eyes and take a deep breath in, filling your lungs completely. Now, slowly release the breath, feeling a wave of relaxation spread throughout your body. Let's do this two more times, inhaling deeply and exhaling slowly.

As you continue to breathe naturally, bring your attention to the top of your head. Imagine a sense of warmth and relaxation beginning to spread from the top of your scalp, flowing down into your forehead and eyebrows. Allow this sensation to release any tension as it moves into your eyes, which feel calm and relaxed.

Now, let this feeling of relaxation flow down into your cheeks and jaw. Unclench your teeth and let go of any tension. Notice how your face feels completely at ease. With each breath you take, this relaxing energy continues to travel down your neck and sho

In [21]:
# user question
user_name = "Nyx"
query = "Give me a relaxing meditation script for addiction."
meditation_length = "15 minutes"
# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(user_name=user_name, chunks_formatted=chunks_formatted, meditation_length=meditation_length, query=query)
answer = llm(prompt_formatted)
print(answer)

Welcome, Nyx. Let's embark on a 15-minute meditation journey designed to bring relaxation and support your recovery from addiction. Please find a comfortable seated position in a quiet space where you won't be disturbed. Close your eyes and allow your hands to rest gently on your knees or in your lap.

Take a deep breath in through your nose, slowly filling your lungs with air. Feel your chest and belly expand. Hold the breath for a moment, and then exhale slowly through your mouth, letting all the tension and stress leave your body. Repeat this deep breathing three times, each time feeling more relaxed.

Now, let your breath return to a natural rhythm. Notice the rise and fall of your chest. Observe the sensation of air flowing in and out of your nostrils. If your mind begins to wander, gently guide your focus back to your breath. This is your anchor, keeping you present and centered.

With each breath, imagine a wave of calm washing over you, from the tips of your toes to the top of 